In [27]:
%use lib-ext(0.11.0-398)

@file:DependsOn("com.brainvault:utils:0.5.0")
import com.brainvault.utils.units.*
import com.brainvault.utils.mechanics.*

# Base info

In [28]:
import kotlin.time.Duration

val maxFeed = 1500.mmPerMinute

val ballBearingSpindlePitch = 5.mm.length
val beltRatio = GearRatio(.25)

val linearAcceleration = 0.5.mps2
val timeToGetUpToSpeed = (maxFeed.inMetersPerSec / linearAcceleration.inMetersPerSec2).seconds

val omegaScrew = maxFeed.toAngularForScrew(ballBearingSpindlePitch)
val omegaMotor = omegaScrew / beltRatio
val alphaMotor = omegaMotor / timeToGetUpToSpeed

val ramMass = 500.kg

val ballscrewEfficiency = 0.95

# Z-axis

In [35]:
DISPLAY(HTML("<h2>Inertia</h2>"))

// Inertia van de klemkoppeling tussen motor en verlengas
// Opgemeten in SolidWorks
//val couplingInertia = 286466.5.gMm2
// https://www.maedler.nl/Article/61588022
val couplingInertia = hollowCylinderInertia(.6.kg, 55.mm.diameter, 22.mm.diameter)
// https://www.maedler.nl/Article/60156619
val coupling2Inertia = hollowCylinderInertia(500.g, 66.mm.diameter, 22.mm.diameter)
// https://www.maedler.nl/Article/60155619
val coupling3Inertia = hollowCylinderInertia(270.g, 56.mm.diameter, 22.mm.diameter)
// Inertia van de verlengas
// Opgemeten in SolidWorks
val axleInertia = 22770.75.gMm2
// Inertia van de z=18 poelie
// Opgemeten in SolidWorks
val z18pulleyInertia = 58615.72.gMm2

// Inertia van de z=72 poelie
val z72pulleyInertia = hollowCylinderInertia(3238.g, 190.mm.diameter, 22.mm.diameter)
// Inertia van de kogelomloopspindel
val spindleInertia = cylinderInertia(6.kg, 40.mm.diameter)
// Inertia van de ram
// 500 kg is mijn eigen schatting
val ramInertia = linearInertia(ramMass, ballBearingSpindlePitch)


val totalInertia = totalInertia(
    couplingInertia to 1.0.ratio,
    axleInertia to 1.0.ratio,
    z18pulleyInertia to 1.0.ratio,
    z72pulleyInertia to beltRatio,
    spindleInertia to beltRatio,
    ramInertia to beltRatio
)

"The total require inertia for the Z-axis is $totalInertia"

"Coupling 1 inertia: ${couplingInertia}. Coupling 2 inertia: ${coupling2Inertia}. Coupling 3 inertia: ${coupling3Inertia}"


Inertia

Coupling 1 inertia: 2,63175e-04 kg⋅m². Coupling 2 inertia: 3,02500e-04 kg⋅m². Coupling 3 inertia: 1,22175e-04 kg⋅m²

In [30]:
DISPLAY(HTML("<h2>Torque</h2>"))

val requiredTorqueForInertia = totalInertia * alphaMotor
DISPLAY("Required torque for inertia: $requiredTorqueForInertia")

val maxCuttingForce = 8.kN
val requiredTorqueForMaxCuttingForce = maxCuttingForce.toTorqueForScrew(ballBearingSpindlePitch, ballscrewEfficiency)
val requiredTorqueForMaxCuttingForceAtMotor = requiredTorqueForMaxCuttingForce * beltRatio

DISPLAY("Required torque for max cutting force: $requiredTorqueForMaxCuttingForceAtMotor")

val totalTorque = requiredTorqueForInertia + requiredTorqueForMaxCuttingForceAtMotor

DISPLAY("Total required torque: ${totalTorque}")



Torque

Required torque for inertia: 3,48868 N·m

Required torque for max cutting force: 1,67532 N·m

Total required torque: 5,16400 N·m

In [31]:
fun MARKDOWN(text: String): MimeTypedResult = MimeTypedResult(mapOf("text/markdown" to text.trimIndent()))

MARKDOWN("""
## Z-axis – Conclusion

We computed the equivalent inertia and torque for the Z-axis (horizontal ram).
SolidWorks was used for coupling, extension shaft, and z=18 pulley; the z=72 pulley, ballscrew and ram were hand-derived.

### Assumptions
- **Extreme case:** linear acceleration **${linearAcceleration}** and cutting force **${maxCuttingForce.toKnString()}**.
- **Simplifications:** belt & small parts inertia, bearing friction ignored.
- **z=72 pulley:** treated as homogeneous (actual part is steel + polymer).
- **Gravity:** not included (horizontal axis / balanced).

### Results
- Total equivalent inertia @ motor: **≈ ${totalInertia}**
- Torque for inertia (α from v→t): **≈ ${requiredTorqueForInertia}**
- Torque for cutting force (8 kN, η=0.95, i=4): **≈ ${requiredTorqueForMaxCuttingForceAtMotor}**
- **Total required torque (peak): ≈ ${totalTorque}**

### Interpretation
These figures reflect a *worst-case* setting for this machine size.
In normal operation (lower accel and/or cutting force) the continuous torque will be well below this peak.

A small 0.75 kW motor (e.g. SGM7J-08A) is under-dimensioned; a ~1.5 kW class (e.g. SGM7A-15A) is a closer fit, or lower accel / higher ratio reduces the requirement significantly.
""")

## Z-axis – Conclusion

We computed the equivalent inertia and torque for the Z-axis (horizontal ram).
SolidWorks was used for coupling, extension shaft, and z=18 pulley; the z=72 pulley, ballscrew and ram were hand-derived.

### Assumptions
- **Extreme case:** linear acceleration **0.5 m/s²** and cutting force **8 kN**.
- **Simplifications:** belt & small parts inertia, bearing friction ignored.
- **z=72 pulley:** treated as homogeneous (actual part is steel + polymer).
- **Gravity:** not included (horizontal axis / balanced).

### Results
- Total equivalent inertia @ motor: **≈ 1,38810e-03 kg⋅m²**
- Torque for inertia (α from v→t): **≈ 3,48868 N·m**
- Torque for cutting force (8 kN, η=0.95, i=4): **≈ 1,67532 N·m**
- **Total required torque (peak): ≈ 5,16400 N·m**

### Interpretation
These figures reflect a *worst-case* setting for this machine size.
In normal operation (lower accel and/or cutting force) the continuous torque will be well below this peak.

A small 0.75 kW motor (e.g. SGM7J-08A) is under-dimensioned; a ~1.5 kW class (e.g. SGM7A-15A) is a closer fit, or lower accel / higher ratio reduces the requirement significantly.